In [7]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

def load_data(data_path, categories):
    data = []
    labels = []
    
    for category in categories:
        folder_path = os.path.join(data_path, category)
        for img_name in os.listdir(folder_path):
            if img_name == '.DS_Store':
                continue
            img_path = os.path.join(folder_path, img_name)
            
            # Read the image and resize it to 224x224 pixels
            img = cv2.imread(img_path)
            if img is None:
                print(f"Could not load image at {img_path}")
                continue
            img = cv2.resize(img, (224, 224))
            
            data.append(img)
            labels.append(category)
    
    return np.array(data), np.array(labels)

In [8]:
def preprocess_labels(labels):
    labels = np.array([0 if label == 'with_mask' else 1 for label in labels])
    return to_categorical(labels)

def normalize_data(data):
    data = np.array(data, dtype="float32")
    return data / 255.0

def create_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(2, 2))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    
    model.add(Dense(50, activation='relu'))
    
    model.add(Dense(2, activation='softmax'))
    
    return model

In [9]:

if __name__ == '__main__':
    data_path = '/Users/iWolfer/Desktop/Pycharm/homework(pds6)/Lessons 60+/66/data'
    categories = ['with_mask', 'without_mask']

    data, labels = load_data(data_path, categories)
    
    labels = preprocess_labels(labels)
    data = normalize_data(data)
    
    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)
    
    input_shape = (224, 224, 3)
    model = create_model(input_shape)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(trainX, trainY, epochs=10, validation_split=0.1, batch_size=32)
    
    def predict_mask(image_path):
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        img = np.reshape(img, [1, 224, 224, 3])
        prediction = model.predict(img)
        
        return 'З маскою' if np.argmax(prediction) == 0 else 'Без маски'

Epoch 1/10
31/31 [==============================] - 12s 367ms/step - loss: 0.5234 - accuracy: 0.8202 - val_loss: 0.1479 - val_accuracy: 0.9455
Epoch 2/10
31/31 [==============================] - 11s 364ms/step - loss: 0.1265 - accuracy: 0.9535 - val_loss: 0.1257 - val_accuracy: 0.9545
Epoch 3/10
31/31 [==============================] - 11s 365ms/step - loss: 0.0697 - accuracy: 0.9737 - val_loss: 0.1323 - val_accuracy: 0.9636
Epoch 4/10
31/31 [==============================] - 11s 367ms/step - loss: 0.0459 - accuracy: 0.9798 - val_loss: 0.0736 - val_accuracy: 0.9818
Epoch 5/10
31/31 [==============================] - 11s 366ms/step - loss: 0.0660 - accuracy: 0.9848 - val_loss: 0.1169 - val_accuracy: 0.9636
Epoch 6/10
31/31 [==============================] - 11s 365ms/step - loss: 0.0937 - accuracy: 0.9687 - val_loss: 0.1278 - val_accuracy: 0.9727
Epoch 7/10
31/31 [==============================] - 11s 364ms/step - loss: 0.0334 - accuracy: 0.9919 - val_loss: 0.1946 - val_accuracy: 0.9455

In [10]:
        
    image_path = '/Users/iWolfer/Desktop/Pycharm/homework(pds6)/Lessons 60+/66/data/without_mask/3.jpg'  
    print(predict_mask(image_path))

1/1 [==============================] - 0s 33ms/step
Без маски
